In [1]:
# 최종 데이터 읽어오기 (변수 5개 (Np, Vp, Bz, Bt, Bl)만 남긴 combine)

import numpy as np
import pandas as pd

np.random.seed(5)

path='C:\Projects\keras_talk\comp_model_data\\'

combine_data = pd.read_csv(path + 'combine_interpolation_an.csv', engine='python') 

kp_data = pd.read_csv(path + 'kp_data.csv') # 지자기교란 지수 파일에서 엑셀 자체로 index부분 없애고 kp_data로 이름 바꿈

In [2]:
from keras.models import Sequential
from keras.layers import Dense, LSTM
from sklearn.preprocessing import MinMaxScaler

Using TensorFlow backend.


In [3]:
def create_dataset(signal_data, result_data, look_back=1):
    dataX, dataY = [], []
    for i in range(0, len(signal_data), look_back):
        dataX.append(signal_data[i:(i+look_back)]) # look_back = 12 단위로 3시간씩 끊어짐 - [0:12],[12:24],...
    for j in range(0, len(result_data)):
        for k in range(0, 8): # 24시간이 3시간 단위로 쪼개져서 총 8개의 column 있음
            dataY.append(result_data[j][k]) # kp지수 데이터의 하나 하나를 뽑아내어 3시간의 combine 데이터와 대응시킴
    return np.array(dataX), np.array(dataY)

look_back = 12

In [4]:
# 1. 데이터셋 생성하기

signal_data = combine_data.values
result_data = kp_data.values

x_train = signal_data[0:385728] # 2920*12*8 + 2928*12*3 (00,04,08년 윤년) = 385728 (99~09년까지의 데이터를 train으로)
x_val = signal_data[385728:455808] # 2920*12*2=70080 (10~11년까지의 데이터를 val으로)
x_test = signal_data[455808:] # 2920*12*1 + 2928*12*1 =70080 (12년 윤년) (12~13년까지의 데이터를 test으로)

y_train = result_data[0:4018] # 99~09년도
y_val = result_data[4018:4748] # 10~11년도
y_test = result_data[4748:] # 12~13년도


# 데이터셋 생성

x_train, y_train = create_dataset(x_train, y_train, look_back)
x_val, y_val = create_dataset(x_val, y_val, look_back)
x_test, y_test = create_dataset(x_test, y_test, look_back)

In [5]:
# 데이터셋 전처리 (RNN은 3차원으로 처리를 해야 한다고 함)
# 변수 5개 (Np, Vp, Bz, Bt, Bl)

x_train = x_train.reshape(x_train.shape[0], x_train.shape[1], 5) # (number, timestep, feature)
x_val = x_val.reshape(x_val.shape[0], x_val.shape[1], 5)
x_test = x_test.reshape(x_test.shape[0], x_test.shape[1], 5)

In [6]:
# 원핫 인코딩 (어떤 값을 리스트 내에 0,1로 표시하는 벡터로 표현하도록 함, 예를 들어 3인 경우엔 [0,0,0,1,0,0,0,0,0,0]으로 표현됨)

from keras.utils import to_categorical

size = 10

y_train = to_categorical(y_train, num_classes=size)
y_val = to_categorical(y_val, num_classes=size)
y_test = to_categorical(y_test, num_classes=size)

In [ ]:
# 2. 모델 구성하기

from keras.optimizers import Adam

Optimizer = Adam(lr = 0.001)

model = Sequential()
model.add(LSTM(48,input_shape=(12,5)))
model.add(Dense(32, activation='relu'))
model.add(Dense(10, activation='softmax'))

# 3. 모델 학습과정 설정하기

model.compile(loss='mean_squared_error', optimizer=Optimizer, metrics=['accuracy'])

# 4. 모델 학습시키기
hist = model.fit(x_train, y_train, epochs=100, batch_size=50, validation_data=(x_val, y_val))

In [9]:
# 5. 모델 평가하기

trainScore = model.evaluate(x_train, y_train, verbose=0)
model.reset_states()
print('Train Score: ', trainScore)
valScore = model.evaluate(x_val, y_val, verbose=0)
model.reset_states()
print('Validataion Score: ', valScore)
testScore = model.evaluate(x_test, y_test, verbose=0)
model.reset_states()
print('Test Score: ', testScore)

Train Score:  [0.06547767443479502, 0.454641612742658]
Validataion Score:  [0.06438208816394414, 0.46472602739726027]
Test Score:  [0.06219835975366643, 0.49042407652584863]


In [11]:
# 6. 모델 사용하기 (final_result_prob_another도 interpolation 후 변수 5개만 뽑아서 problem_interpolation1로 만듦)

path='C:\Projects\keras_talk\comp_model_data\\'

xhat = pd.read_csv(path + 'problem_interpolation1.csv', engine='python')

input_data = xhat.values

#input_data = input_data.reshape(input_data.shape[0], input_data.shape[1], 1)

#input_data = input_data.reshape(2920, 12, 5) - 이 두 줄이 필요한지 안한지는 실행해봐야 알 것 같음.

forecast_result = model.predict(input_data)

In [ ]:
# 예측한 kp지수를 엑셀로 다시 정리

result = []

for i in range(0,2920):
    result.append(np.argmax(forecast_result[i]))

final_result = []

for i in range(0, len(result), 8):
    final_result.append(result[i:i+8])

kp_0h = []
kp_3h = []
kp_6h = []
kp_9h = []
kp_12h = []
kp_15h = []
kp_18h = []
kp_21h = []

for i in range(0, len(final_result)):
    kp_0h.append(final_result[i][0])
    kp_3h.append(final_result[i][1])
    kp_6h.append(final_result[i][2])
    kp_9h.append(final_result[i][3])
    kp_12h.append(final_result[i][4])
    kp_15h.append(final_result[i][5])
    kp_18h.append(final_result[i][6])
    kp_21h.append(final_result[i][7])

from collections import OrderedDict

raw_data = OrderedDict()

raw_data['kp_0h'] = kp_0h
raw_data['kp_3h'] = kp_3h
raw_data['kp_6h'] = kp_6h
raw_data['kp_9h'] = kp_9h
raw_data['kp_12h'] = kp_12h
raw_data['kp_15h'] = kp_15h
raw_data['kp_18h'] = kp_18h
raw_data['kp_21h'] = kp_21h

kp_predict = pd.DataFrame(raw_data)

kp_predict.to_csv('./kp_predict.csv', index=False)